# SEGMENTING AND CLUSTERING NEIGHBORHOODS OF TORONTO

### 1 Importing Libraries

In [1]:
import numpy as np            # for efficient numerical operations
import pandas as pd           # for data frame analytics
import requests               # for pulling html pages and creating html documents
from bs4 import BeautifulSoup # for scraping html documents

### 2 Parsing the Wikipedia page on Toronto postal codes

By means of the <i>requests</i> library which has been loaded above, the link provided in the description of the assignment can be employed to retrieve the html page pointed to be that link. This is acchieved via the <i>requests.get()</i> function. Once the html page is retrieved the underlying html document stored in the <i>text</i> attribute of the page object. Lastly, this hmtl document is parsed by means of the <i>BeautifulSoup</i> library.

In [2]:
html_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_page = requests.get(html_link)
html_doc = html_page.text
soup = BeautifulSoup(html_doc,'html.parser')

### 3 Scraping the html document

#### 3.1 Searching the postal code table in the html document and creating a dataframe from it
Firstly, the soup object is searched for the table by means of the <i>find()</i> function and specifying the html tag <i>table</i>. With the help of that same function the rows (html tagged <i>tr</i>) are then extracted from the table, and headers (html tagged <i>th</i>) are extracted from the first row. These headers, together with the number of rows are employed to create a raw pandas dataframe <i>table_df</i> which contains all rows of the table found on the wikipedia page.

In [100]:
table = soup.find('table')
rows = table.findAll('tr')
header = rows[0].findAll('th')
col_num = len(header)
row_num = len(rows)-1

columns = list()
for c in range(col_num):
    if c != col_num-1:
        columns.append(header[c].string.lower())
    else:
        columns.append(header[c].string[:-1].lower())

table_df = pd.DataFrame(columns=columns,index=range(row_num))

for i in range(row_num):
    row = rows[i+1].findAll('td')
    for c in range(col_num):
        table_df.iloc[i,c] = row[c].string
        if table_df.iloc[i,c] == None:
            table_df.iloc[i,c] = row[c].a.string
        
table_df.head()

,postcode,borough,neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### 3.2 Manipulating the raw dataframe
Next, those rows of the raw dataframe <i>table_df</i> data frame who have no borough assigned, are dropped. Next, if among the remaining entries the neighborhood is not assigned, the neighborhood name will be the same as the borough name.

In [101]:
for i in table_df.index:
    if table_df.loc[i,'borough'] == 'Not assigned':
        table_df = table_df.drop(i,axis=0)
    elif table_df.loc[i,'neighbourhood'] == 'Not assigned':
        table_df.loc[i,'neighbourhood'] = table_df.loc[i,'borough']
        
table_df.head()

,postcode,borough,neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


#### 3.3  Creating a clean postal code dataframe from the raw data frame
Lastly, from the raw dataframe <i>table_df</i> the unique postal codes are collected and employed to create a clean dataframe <i>code_df</i> which lists codes, borough names and all associated neighborhoods.

In [124]:
code_list = list(table_df['postcode'].unique())
code_df = pd.DataFrame(columns=columns,index=range(len(code_list)))

for i in code_df.index:
    code = code_list[i]
    sub_df = table_df[table_df['postcode']==code]
    hood_string = ''
    for j in range(sub_df.shape[0]):
        hood_string = hood_string + sub_df.loc[sub_df.index[j],'neighbourhood'] + ', '
    hood_string = hood_string[:-2]
    code_df.loc[i,'postcode'] = code
    code_df.loc[i,'borough'] = sub_df.loc[sub_df.index[0],'borough']
    code_df.loc[i,'neighbourhood'] = hood_string
    
code_df.sort_values('postcode',inplace=True)
    
code_df

,postcode,borough,neighbourhood
6,M1B,Scarborough,"Rouge, Malvern"
12,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
18,M1E,Scarborough,"Guildwood\n, Morningside, West Hill"
22,M1G,Scarborough,Woburn
26,M1H,Scarborough,Cedarbrae
32,M1J,Scarborough,Scarborough Village
38,M1K,Scarborough,"East Birchmount Park\n, Ionview, Kennedy Park"
44,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
51,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West\n"
58,M1N,Scarborough,"Birch Cliff, Cliffside West\n"
